In [ ]:
# import library
from mlxtend.frequent_patterns import apriori as mlx_apriori, association_rules
import pandas as pd
import numpy as np

In [ ]:
# read the file
trn = pd.read_excel("C:\\Users\\sales.co\\Documents\\R\\datatest_product.xlsx")

In [ ]:
# check summary of Quantity
trn.describe()

In [ ]:
# view the data structure
trn.style

In [ ]:
# check any missing values
trn.isna().count()

In [ ]:
# extract the columns name
trn.columns

In [ ]:
#trn.set_index(['invoice_date'], inplace=True)

In [ ]:
# check firt five transaction
trn.head()

In [ ]:
# Pivot the categorical column from row to column and reset the index
df_unstacked = trn.pivot(index="invoice_date", columns="product_name", values="qty").reset_index().rename_axis(columns=None)

In [ ]:
# fill zero for na
df_unstacked = df_unstacked.fillna(0)

In [ ]:
#df_stack=trn.drop(columns="qty", axis =1)

In [ ]:
# check last 5 transaction 
df_unstacked.tail()

In [ ]:
# Slice the columns
data = df_unstacked.iloc[:,1:]

In [ ]:
# Apply True or False to all variables if availabe quantity
df_wide = data.applymap(lambda x: 1 if x >= 1 else 0)

In [ ]:
# sample from df_wide datafram by 3 %
df_sample = df_wide.sample(frac=0.03)

In [ ]:
# filter data which has more than 1 unique value for better market basket analysis
df_wide_sample = df_sample.loc[:, df_sample.apply(lambda x: x.nunique() > 1)].reset_index()

In [ ]:
# Apply True or False to all variables if availabe quantity
df_wide_unique = df_wide_sample.applymap(lambda x: True if x >= 1 else False)

In [ ]:
# Slice the columns
df_sample_unique = df_wide_unique.iloc[:,1:]

In [ ]:
# view first five transaction
df_sample_unique.head()

In [ ]:
# print data structure and datatype
print(df_sample_unique.info())

In [ ]:
#from datetime import datetime, timedelta

# # Get the current date
# current_date = datetime.now().date()

# # Subtract 30 days
# result_date = current_date - timedelta(days=30)
# formatted_result_date = result_date.strftime('%Y-%m-%d')

In [ ]:
# Filter rows based on the condition
#df_unstacked_filtered = df_unstacked[df_unstacked['invoice_date'] >= df_unstacked['invoice_date'].max()-timedelta(days=30)]

In [ ]:
#df_unstacked_filtered

In [ ]:
# import fpgrowth library
from mlxtend.frequent_patterns import fpgrowth

In [ ]:
# apply fpgrowth model with min support 40%
frequent_itemsets = fpgrowth(df_sample_unique, min_support=0.51,use_colnames=True)

print(frequent_itemsets)

In [ ]:
# create function to know relationship with each products
def get_recommendations(df, item):
    """Generate a set of product recommendations using item-based collaborative filtering.
    
    Args:
        df (dataframe): Pandas dataframe containing matrix of items purchased.
        item (string): Column name for target item. 
        
    Returns: 
        recommendations (dataframe): Pandas dataframe containing product recommendations. 
    """
    
    recommendations = df.corrwith(df[item])
    recommendations.dropna(inplace=True)
    recommendations = pd.DataFrame(recommendations, columns=['correlation']).reset_index()
    recommendations = recommendations.sort_values(by='correlation', ascending=False)
    
    return recommendations

In [ ]:
# check bikanery Bhujia releationship with other products
recommendations = get_recommendations(df_sample_unique, 'BIKANERI BHUJIA')
recommendations.head(10)

In [ ]:
# check bikanery sweet farali chivda with other products
recommendations = get_recommendations(df_sample_unique, 'MASALA PONGA')
recommendations.head(10)

In [ ]:
# apply association rules and sort with confidence highest
association_rules(frequent_itemsets).sort_values('confidence', ascending=False)

In [ ]:
# apply association rules and sort with confidence lowest
association_rules(frequent_itemsets).sort_values('confidence', ascending=True)